In [1]:
#20240430

In [ ]:
'''
지금까지는 변수로 스칼라를 사용하였다.

텐서사용하기:
-머신러닝 데이터로 벡터나 행렬 등의 텐서가 주로 사용됨
-텐서 사용시의 주의점 파악과 DeZero 확장을 준비
-지금까지 구현한 DeZero 함수들이 텐서도 다룰 수 있음을 보여줌

* 지금까지 구현한 DeZero 함수에 텐서를 건네면 텐서의 원소마다 스칼라로 계산한다.
텐서의 원소별 스칼라 계산이 이루어지면 스칼라를 가정해 구현한 역전파는 텐서의 원소별 계산에서도 성립된다.
마지막 출력이 스칼라인 계산 그래프에 대한 역전파를 다루기 때문에 상관없는 것이다.

'''

In [10]:
import os, sys
#os.getcwd()
sys.path.append("C:\\Users\\user\\1")

from dezero import Variable
import numpy as n
import dezero.functions as F


In [12]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
c = Variable(np.array([[10,20,30],[40,50,60]]))
t = x + c
y = F.sum(t)
print(y)

variable(231)


In [13]:
y.backward(retain_grad=True)
print(y.grad)
print(t.grad)
print(x.grad)
print(c.grad)

variable(1)
variable([[1 1 1]
          [1 1 1]])
variable([[1 1 1]
          [1 1 1]])
variable([[1 1 1]
          [1 1 1]])


In [14]:
'''
<원소별로 계산하지 않는 함수>
그렇다면 원소별로 계산하지 않는 함수는 어떻게 처리해야할까? -> 텐서의 형상을 변환하는 함수를 구현
- 텐서의 형상을 변환하는 reshape 함수
- 행렬을 전치하는 transpose 함수
- 두 함수 모두 텐서의 형상을 바꾸는 함수

'''

'\n<원소별로 계산하지 않는 함수>\n그렇다면 원소별로 계산하지 않는 함수는 어떻게 처리해야할까? -> 텐서의 형상을 변환하는 함수를 구현\n- 텐서의 형상을 변환하는 reshape 함수\n- 행렬을 전치하는 transpose 함수\n- 두 함수 모두 텐서의 형상을 바꾸는 함수\n\n'

In [15]:
#reshape 함수 구현
import numpy as np

x = np.array([[1,2,3],[4,5,6]])
y = np.reshape(x, (6,)) #x의 형상을 (2,3)에서 (6,)으로 변환. 원소 수는 같고 형상만 바뀜
print(y)

[1 2 3 4 5 6]


In [16]:
'''
reshape 함수는 단순히 형상만 변환한다. 구체적인 계산은 아무것도 하지 않는다는 것이다.
따라서 역전파는 출력 쪽에서 전해지는 기울기에 아무런 손도 대지 않고 입력 쪽으로 흘려보내준다.
이것이 reshape 함수의 역전파이다.
'''

'\nreshape 함수는 단순히 형상만 변환한다. 구체적인 계산은 아무것도 하지 않는다는 것이다.\n따라서 역전파는 출력 쪽에서 전해지는 기울기에 아무런 손도 대지 않고 입력 쪽으로 흘려보내준다.\n이것이 reshape 함수의 역전파이다.\n'

In [22]:
'''
DeZero용 reshape 함수 구현
- Reshape 클래스를 초기화할 때 변형 목표가 되는 형상을 shape 인수로 받음
- 순전파시에 넘파이의 reshape 함수를 사용하여 형상을 변환
- self.x_shape = x.shape 코드에서 입력 x의 형상을 기억해 둠
- 역전파에서 입력 형상 (self.x_shape)로 변환할 수 있음

'''
from dezero.core import Function
class Reshape(Function):
    def __init__(self, shape):
        self.shape = shape

    def forward(self, x):
        self.x_shape = x.shape
        y = x.reshape(self.shape)
        return y

    def backward(self, gy):
        return reshape(gy, self.x_shape)

In [23]:
from dezero.core import as_variable

def reshape(x, shape):
    if x.shape == shape:
        return as_variable(x)
    return Reshape(shape)(x)

In [24]:
'''
구현한 reshape 함수 사용
- reshape 함수를 사용하여 형상을 변환시킴
- y.backward(retain_grad=True)를 수행하여 x의 기울기를 구함
- 이 과정에서 y의 기울기도 자동으로 채워짐
- 채워진 기울기의 형상은 y와 같음 (y.grad.shape == y.shape)
- 원소는 모두 1로 이루어진 텐서임
'''

'\n구현한 reshape 함수 사용\n- reshape 함수를 사용하여 형상을 변환시킴\n- y.backward(retain_grad=True)를 수행하여 x의 기울기를 구함\n- 이 과정에서 y의 기울기도 자동으로 채워짐\n- 채워진 기울기의 형상은 y와 같음 (y.grad.shape == y.shape)\n- 원소는 모두 1로 이루어진 텐서임\n'

In [25]:
import numpy as np
from dezero import Variable
import dezero.functions as F

x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.reshape(x, (6,))
y.backward(retain_grad=True)
print(x.grad)

variable([[1 1 1]
          [1 1 1]])


In [26]:
#Variable에서 넘파이의 reshape 사용
x = np.random.rand(1,2,3)

y = x.reshape((2,3)) #튜플로 받기
y = x.reshape([2,3]) #리스트로 받기
y = x.reshape(2,3) #인수를 그대로 풀어서 받기

In [27]:
#DeZero의 reshape 함수를 넘파이의 reshape와 비슷하게 만들기
'''
- Variable 클래스에 가변 인수를 받는 reshape 메서드를 추가
- reshape 함수를 Variable 인스턴스의 메서드 형태로 호출할 수 있음
'''

'\n- Variable 클래스에 가변 인수를 받는 reshape 메서드를 추가\n- reshape 함수를 Variable 인스턴스의 메서드 형태로 호출할 수 있음\n'

In [31]:
#CORE파일의 Variable에 가변인수를 받는 reshape 메소드 추가.
    # def reshape(self, *shape):
    #     if len(shape) == 1 and isinstance(shape[0], (tuple, list)):
    #         shape = shape[0]
    #     return dezero.functions.reshape(self, shape)

In [33]:
x = Variable(np.random.randn(1,2,3))
y = x.reshape((2,3))
y = x.reshape(2,3)

In [34]:
#행렬의 전치
'''
행렬을 전치하면 행렬의 형상이 바뀜 (ex.가로 ==(2x3)에서 ;;(3x2)로  변경) 
전치를 transpose라는 이름의 DeZero함수로 구현
'''

In [36]:
#넘파이에서 transpose 함수를 사용하여 전치
x = np.array([[1,2,3],[4,5,6]])
y = np.transpose(x)
print(y)

[[1 4]
 [2 5]
 [3 6]]


In [38]:
#따라서 역전파에서는 출력 쪽에서 전해지는 기울기의 형상만 변경
#DeZero의 TRANSPOSE 함수는 다음처럼 구현

class Transpose(Function):
    def forward(self, x):
        y = np.transpose(x)
        return y

    def backward(self, gy):
        gx = transpose(gy)
        return gy

In [40]:
#transpose 함수 사용
x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.transpose(x)
y.backward()
print(x.grad)

variable([[1 1 1]
          [1 1 1]])


In [42]:
#Variable 인스턴스에서도 transpose 함수를 사용할 수 있도록 코드 추가
'''
def transpose(self):
       return dezero.functions.transpose(self)  첫 번째인 transpose는 인스턴스 메서드로 이용하기 위한 코드
    @property
    def T(self):
        return dezero.functions.transpose(self) 두 번째 T에는 @property 데코레이터가 붙어 인스턴수 변수로 사용
'''

'\ndef transpose(self):\n       return dezero.functions.transpose(self)\n    @property\n    def T(self):\n        return dezero.functions.transpose(self)\n'

In [44]:
#더 범용적으로 사용할 수 있는 np.transpose 함수
A, B, C, D = 1,2,3,4
x = np.random.rand(A,B,C,D)
y = x.transpose(1,0,3,2)
'''
축의 순서를 지정하면 그에 맞게 데이터의 축이 달라진다. 인수를 None으로 주면 축이 역순으로 정렬됨. 
x가 행렬일 때 x.transpose()를 실행하면 0번째와 1번째 축의 데이터가 1번째와 0번째 순서로 바뀐다. 행렬이 전치되는 것이다.
'''

'\n축의 순서를 지정하면 그에 맞게 데이터의 축이 달라진다. 인수를 None으로 주면 축이 역순으로 정렬됨. \nx가 행렬일 때 x.transpose()를 실행하면 0번째와 1번째 축의 데이터가 1번째와 0번째 순서로 바뀐다. 행렬이 전치되는 것이다.\n'

In [ ]:
#합계함수
#DeZero에 합계를 구하는 함수 sum 구현
'''
<sum함수의 역전파>
덧셈의 미분: 덧셈을 수행한 후 변수 y로부터 역전파함. 역전파는 출력 쪽에서 전해지는 기울기를 그대로 입력 쪽으로 흘려보냄.
        x0와 x1에는 출력 쪽에서 전해준 1이라는 기울기를 두 개로 복사하여 전달
* 원소가 2개로 구성된 벡터 합의 역전파
벡터에 sum 함수를 적용하면 스칼라를 출력함
역전파는 출력 쪽에서 전해준 값인 1을 [1, 1]이라는 벡터로 확장해 전파함

* 원소가 2개 이상인 벡터의 합에 대한 역전파 
기울기 벡터의 원소 수만큼 복사하면 됨
기울기를 입력 변수의 형상과 같아지도록 복사함
입력 변수가 2차원 이상의 배열일 때도 동일하게 적용됨


<DeZero의 Sum 클래스와 sum 함수 구현>
sum 함수 역전파에서는 입력 변수의 형상과 같아지도록 기울기의 원소를 복사함.
복사작업은 지정한 형상에 맞게 원소를 복사하기 위해 broadcast_to 함수 사용.
(broadcast_to 함수: Variable 인스턴스인 x의 원소를 복사하여 shape 인수로 지정한 형상이 되도록 만들어 주는 함수)
broadcast_to 함수를 사용하여 입력 변수와형상이 같아지도록 기울기 gy의 원소를 복사함. 행렬을 입력하여 벡터가 아닌 경우의 동작 확인.

'''

In [47]:
#sum함수 구현
class Sum(Function):
    def forward(self, x):
        self.x_shape = x.shape
        y = x.sum()
        return y

    def backward(self, gy):
        gx = broadcast_to(gy, self.x_shape)
        return gx

def sum(x):
    return Sum()(x)

In [48]:
#sum함수 사용해보기
import numpy as np
from dezero import Variable
import dezero.functions as F

x = Variable(np.array([1, 2, 3, 4, 5, 6]))
y = F.sum(x)
y.backward()
print(y)
print(x.grad)

variable(21)
variable([1 1 1 1 1 1])


In [49]:
#sum함수 사용해보기2
import numpy as np
from dezero import Variable
import dezero.functions as F

x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
y = F.sum(x)
y.backward()
print(y)
print(x.grad)


variable(21)
variable([[1 1 1]
          [1 1 1]])


In [50]:
#axis와 keepdims- 합계를 구할 때 축을 지정하기
'''
axis(축) 지정 인수
- 합계를 구할 때 축을 지정할 수 있음
- x의 형상은 (2, 3) 이고 출력 y의 형상은 (3, )임
- Axis는 축을 뜻하며, 다차원 배열에서 화살표의
방향을 의미함

keepdims 인수
- keepdims 는 입력과 출력의 차원 수(축 수)를 똑같게 유지할지 정하는 플래그
- keepdims=True 로 지정하면 축의 수가 유지
- keepdims=False로 지정하면 y의 형상은 스칼라

'''
x = np.array([[1,2,3],[4,5,6]])
y = np.sum(x, axis=0)
print(y)
print(x.shape, '->', y.shape)

'''
axis는 축이라는 뜻으로 다차원 배열에서 화살표의 방향을 의미한다.
axis=1 (가로) axis=0(세로)
x.sum(axis=1) 는 세로를 모두 sum, 
x.sum(axis=0) 는 가로를 모두 sum
'''

[5 7 9]
(2, 3) -> (3,)


In [51]:
'''
DeZero의 sum함수를 두 인수를 지원하게 만들기
Sum 클래스를 초기화할 때 axis와 keepdims를 입력받아 속성으로 설정한다.
순전파에서는 이 속성들을 사용해 합계를 구한다. 
역전파에서는 기존처럼 broadcast_to 함수를 사용하여 입력변수의 형상과 같아지도록 기울기의 원소를 복사한다.
'''

x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.sum(x,axis=0)
y.backward()
print(y)
print(x.grad)

x = Variable(np.random.rand(2,3,4,5))
y = x.sum(keepdims=True)
print(y.shape)

variable([5 7 9])
variable([[1 1 1]
          [1 1 1]])
(1, 1, 1, 1)


In [ ]:
'''
<브로드캐스트 함수>
- 서로 다른 형상(shape)을 가진 배열들간에 산술 연산을 수행하기 위해 배열의 형상을 조정
- 브로드캐스트를 사용하여 작은 배열을 큰 배열에 맞추어 연산을 수행할 수 있음

<넘파이의 브로드케스트 함수>
- 넘파이의 np.broadcast_to(x, shape) 함수(ndarray 인스턴스인 x의 원소를 복제하여 shape 인수로 지정한 형상이 되도록 해줌)
- 원래는 (3, ) 형상이던 1차원 배열의 원소를 복사하여 (2, 3) 형상으로 바꿈

< np.broadcast_to 함수의 역전파>
- 입력 x의 형상과 같아지도록 기울기의 합을 구함
- sum_to( x, shape) 함수가 있으면 간단하게 해결됨 (x의 원소의 합을 구해 shape 형상으로 만들어주는 함수)


'''

In [52]:
x = np.array([1,2,3])
y = np.broadcast_to(x,(2,3))
print(y)

[[1 2 3]
 [1 2 3]]


In [ ]:
'''
<넘파이 버전 sum_to 함수 준비>
- 파일 위치는 dezero/utils.py
- sum_to(x, shape) 함수는 shape 형상이 되도록 합을 계산
- 기능은 np.sum 함수와 같지만 인수를 주는 방법이 다르다.

<sum_to 함수의 역전파>
- 역전파는 broadcast_to 함수를 그대로 이용
- broadcast_to 함수를 사용하여 입력 x의 형상과 같아지도록 기울기의 원소를 복제함

'''

In [53]:
from dezero.utils import sum_to

x = np.array([[1,2,3],[4,5,6]])
y = sum_to(x, (1,3))
print(y)

y = sum_to(x, (2,1))
print(y)

[[5 7 9]]
[[ 6]
 [15]]


In [ ]:
'''
<BroadcastTo 클래스와 broadcast_to 함수 구현>
- 역전파에서는 입력 x와 형상을 일치시키는 데 DeZero의 sum_to 함수를 이용함
- SumTo 클래스와 sum_to 함수 구현
- 역전파에서는 입력 x와 형상이 같아지도록 기울기의 원소를 복제함 (이를 위해 DeZero 의 broadcast_to 함수를 사용)
- broadcast_to 함수와 sum_to 함수는 상호 의존적임
(순전파일때는 sum_to, 역전파일때는 broadcast)

<브로드캐스트란>
- 형상이 다른 다차원 배열끼리의 연산을 가능하게 하는 넘파이 기능
- sum_to 함수를 구현한 이유는 넘파이 브로드캐스트에 대응하기 위함
'''

In [55]:
x0 = np.array([1,2,3])
x1 = np.array([10])
y = x0 + x1    #x0와 x1은 형상이 다르지만, 계산 과정에서 x1의 원소가 x0 형상에 맞춰 복제됨
print(y)

[11 12 13]


In [57]:
#DeZero에서의 브로드캐스트
x0 = Variable(np.array([1,2,3]))
x1 = Variable(np.array([10]))
y = x0 + x1
print(y)

'''
순전파는 ndarray 인스턴스를 사용해 구현했기 때문에 브로드캐스트가 일어남
브로드캐스트의 역전파는 일어나지 않음
넘파이 브로드캐스트는 broadcast_to 함수에서 이루어지고, 
broadcast_to 함수의 역전파는 sum_to 함수에서 일어남
'''

variable([11 12 13])


'\n순전파는 ndarray 인스턴스를 사용해 구현했기 때문에 브로드캐스트가 일어남\n브로드캐스트의 역전파는 일어나지 않음\n넘파이 브로드캐스트는 broadcast_to 함수에서 이루어지고, \nbroadcast_to 함수의 역전파는 sum_to 함수에서 일어남\n'

In [59]:
#브로드 캐스트 역전파 계산
x0 = Variable(np.array([1,2,3]))
x1 = Variable(np.array([10]))
y = x0 + x1
print(y)

y.backward()
print(x1.grad) #브로드캐스트가 일어나도 DeZero가 올바르게 동작함을 확인

variable([11 12 13])
variable([3])


In [60]:
'''
<행렬의 곱>
41단계의 주제는 벡터의 내적과 행렬의 곱

1) 벡터의 내적: 두 벡터 사이으이 대응 원소의 곱을 모두 합한 값이 벡터의 내적
2) 행렬의 곱: 왼쪽 행렬의 가로방향 벡터와 오른쪽 행렬의 세로방향 벡터 사이의 내적을 계산. 그리고 그 결과가 새로운 행렬의 원소가 됨.

'''

'\n<행렬의 곱>\n41단계의 주제는 벡터의 내적과 행렬의 곱\n\n1) 벡터의 내적: 두 벡터 사이으이 대응 원소의 곱을 모두 합한 값이 벡터의 내적\n2) 행렬의 곱: 왼쪽 행렬의 가로방향 벡터와 오른쪽 행렬의 세로방향 벡터 사이의 내적을 계산. 그리고 그 결과가 새로운 행렬의 원소가 됨.\n\n'

In [62]:
#벡터의 내적과 행렬의 곱을 넘파이를 사용하여 구현 (np.dot 함수 사용)
import numpy as np

#벡터의 내적
a = np.array([1,2,3])
b = np.array([4,5,6])
c = np.dot(a,b)
print(c)

#행렬의 곱
a = np.array([[1,2],[3,4]])
b = np.array([[5,6],[7,8]])
c = np.dot(a,b) #np.dot(x,y)의 두 인수가 모두 1차원 배열이면 벡터의 내적을 계산하고 인수가 2차우너 배열이면 행렬의 곱을 계산한다.
print(c)

32
[[19 22]
 [43 50]]


In [ ]:
'''
행렬과 벡터를 사용할때 주의해야할 점 -> 행렬의 형상 체크
- 행렬 a와 b의 대응하는 차원(축)의 원소 수가 일치해야 함
- 결과로 만들어진 행렬 c의 형상은 행렬 a와 같은 수의 행을, 행렬 b와 같은 수의 열을 가짐.
- 행렬의 곱에서는 대응하는 축의 원소 수를 일치시킨다.

<행렬 곱의 역전파>

- 최종적으로 스칼라값인 L을 출력하는 계산을 다룬다.
- L의 각 변수에 대한 미분을 역전파로 구함

예시) y =xW라는 계산을 예로 행렬 곱의 역전파를 설명.
x, W, y의 형상은 1xD, DxH, 1xH. 

x의 i번째 원소에 대한 미분은 𝜕𝐿/𝜕𝑥𝑖 = ∑(J) 𝜕𝐿/𝜕𝑦j X 𝜕𝑦j/𝜕𝑥𝑖

- 𝑥𝑖 를 변화시키면 벡터 y의 모든 원소가 변함
- y의 각 원소의 변화를 통해 궁극적으로 L이 변화하게 됨
- 𝑥𝑖 에서 L에 이르는 연쇄 법칙의 경로는 여러 개 있고, 그 총합이 𝜕𝐿/𝜕𝑥𝑖임


'''